In [1]:
from __future__ import print_function

In [2]:
# возвращает True если в графе для каждого ребра u->v есть соответствующее ребро v->u
def check_graph_is_ordered(inp):
    all_edges = {}
    for u,v,_ in inp:
        all_edges[(u,v)] = True
    
    for u,v,_ in inp:
        if not all_edges.get( (v,u), False):
            return False
        
    return True

# Read graph, convert words to ids

In [11]:
lines = open('watset/data/ru/edges.count.txt').read().strip().split('\n')
inp = map(lambda t : t.split('\t'), lines)
print(len(inp))

425420


In [4]:
check_graph_is_ordered(inp)

True

In [12]:
_words2id = {}
_id2words = {}
_N = 0

def word2id(word):
    global _N
    i = _words2id.get(word)
    if i is None:
        _words2id[word] = _N
        _id2words[_N] = word
        i = _N
        _N += 1
    return i

def id2word(i):
    return _id2words[i]

In [13]:
from collections import defaultdict
reb = defaultdict(list)

for u, v, w in inp:
    u, v, w = word2id(u), word2id(v), int(w)
    reb[u].append( (v, w) )


# get EGO graphs and clusterize them

In [7]:
def make_ego(u):
    V = set(map(lambda x: x[0], reb[u]))
    ego_reb = {}
    for v in V:
        ego_reb[v] = filter(lambda edge: edge[0] in V, reb[v])
        
    return ego_reb

In [1]:
def argmax(items, default_value = 0.0):
    best_arg, best_value = None, default_value
    for arg, value in items:
        if value > best_value:
            best_arg, best_value = arg, value
    return best_arg


In [2]:
import random

def chinise_whispers(reb, max_steps = 10):
    V = reb.keys()
    classes = { u : i for i, u in enumerate(V) }
    for i in range(max_steps):
        random.shuffle(V)
        for u in V:
            cnt = defaultdict(float)
            for v, w in reb[u]:
                cnt[classes[v]] += w # TODO: change this for other measures
            
            classes[u] = argmax(cnt.items(), default_value=-1.0)

    clusters = defaultdict(list)
    for u, cls in classes.items():
        clusters[cls].append(u)
        
    return clusters.values()

In [9]:
senses = {}
ctx = {}

for u in reb.keys():
    ego_reb = make_ego(u)
    weights = dict(reb[u])
    #print(id2word(u), ego_reb)
    clusters = chinise_whispers(ego_reb)
    
    senses[u] = []
    for i, cluster in enumerate(clusters):
        s = u*1000 + i
        senses[u].append(s)
        ctx[s] = {v : weights[v] for v in cluster}

In [10]:
len(reb), len(senses), len(ctx)

(83588, 83588, 111458)

In [11]:
import math

# ctx1, ctx2 can be basically
# a) dict with fast lookup by key on ctx1 and ctx2
# b) list of pairs, sorted by keys
# this function is for dicts
def cos_sim(ctx1, ctx2):
    dot_product = 0
    
    sum1 = 0
    sum2 = 0
    
    for u, weight in ctx1.items():
        dot_product += weight * ctx2.get(u, 0)
        sum1 += weight**2
        
    for _, weight in ctx2.items():
        sum2 += weight**2
        
    return dot_product / math.sqrt(sum1 * sum2)

In [ ]:
# depends on senses[], ctx[]
def disambiguate_word(v, context):
    similarities = [sense_v, cos_sim(context, ctx[sense_v]) for sense_v in senses[v]]
    return argmax(similarities)

In [10]:
ctx_new = {}

for s, curr_ctx in ctx.items():
    context = dict(curr_ctx)
    context[s // 1000] = 1 # hack to add current word to context as in disambiguate.py
    
    ctx_new[s] = { disambiguate_word(v, context) : weight for v, weight in curr_ctx.items() }
    

NameError: name 'ctx' is not defined

In [13]:
len(ctx_new.items()), len(reb), len(senses), len(ctx)

111458

In [15]:
for x in ctx_new.keys():
    ctx_new[x] = ctx_new[x].items()

In [16]:
clusters = chinise_whispers(ctx_new)

# Result output

In [17]:
def cluster2str(cluster):
    return '{0}\t{1}'.format(len(cluster), ', '.join( id2word(v // 1000) + str(v% 1000) for v in cluster) )
    

In [18]:
clusters.sort(key=len, reverse=True)


In [19]:
for c in clusters[50:60]:
    print(cluster2str(c))

70	вдовень0, ни_душой_ни_телом0, водерень0, на_хуй0, до_корня0, на_три_буквы0, вдокон0, в_конец0, вдрызг0, абсолютно0, цельно0, куда_подальше0, всесторонне1, совсем0, далеко_и_надолго0, ровно2, в_сторонку0, напрочь0, отнюдь1, в_пизду2, к_чёрту0, к_чёрту1, на_хуй1, радикально1, всюду1, сполна0, напротив3, начисто1, во_всем_объеме0, ни_сном_ни_духом0, на_хрен0, с_какой_целью2, насовсем1, полно2, решительно0, решительно3, полноценно0, вконец0, в_лоск0, все6, вдосталь1, вовсе1, всесторонне0, коренным_образом1, кругом0, многосторонне0, с_головы_до_пят1, целиком0, к_дьяволу0, твердо0, радикально0, вконец1, бесповоротно0, наголову0, полностью0, от_начала_до_конца0, диаметрально0, разносторонне0, ко_всем_чертям0, на_хер0, вполне0, окончательно1, ни_сном_ни_духом1, в_корень0, далеко_и_без_куска_хлеба0, совершенно3, ровным_счётом0, всецело0, в_пизду1, кардинально0
69	убеждённость1, прозорливость0, взгляд1, воззрение1, коллегия2, догадка3, мировоззрение0, мировоззрение1, усмотрение0, синедрион2, 